# Policy Iteration

Le Policy Iteration est l'un des deux algorithmes les plus utilisés pour résoudre un MDP dont les dynamiques ($p(s'|s,a$)) et les rewards ($R_s^a$) sont connus. </br>
L'algorithme de policy iteration alterne autant de fois que nécessaire l'évaluation et l'improvement d'une policy $\pi$, afin de trouver la solution du MDP $\pi_*$.

<img src="img/schema_policyiteration.png" width="250" height="250" align="center"/>

<center><em><span style="color:gray">Sutton and Barto, chapitre 4</span></em></center>

### Evaluation

L'évaluation consiste à calculer la value fonction $V$ de la policy $\pi$ pour tous les states $s$ :

\begin{align}
\large V(s) \; & \large \leftarrow \sum_{a} \pi(a|s) \sum_{s'} p(s'|s,a) [R_{ss'}^a + γ V(s')] \quad \forall s
\end{align}

### Improvement

L'improvement assigne à la policy $\pi$ pour chaque state $s$ la meilleure action par rapport à sa value function $V$. Autrement dit, elle assigne l'action $a$ qui maximize $[R_s^a + γ \sum_{s'} p(s'|s,a) V(s')]$.

\begin{align}
\large \pi(s) \; & \large \leftarrow argmax_a \; Q(s, a) \quad \forall s
\\
& \large \leftarrow argmax_a \; [\sum_{s'} p(s'|s,a) [R_{ss'}^a + γ V(s')]] \quad \forall s
\end{align}

### Iteration

L'algorithme de policy iteration consiste à alterner entre l'évaluation et l'improvement jusqu'à convergence : 
* <strong>Evaluation</strong> : calculer `V_new`. Si la nouvelle évaluation `V_new` est suffisament proche (déterminé par $\theta$) de la précédente `V`, l'aglorithme s'arrête
* <strong>Improvement</strong> : $\pi = greedy(V)$ 

Le pseudocode de l'algorithme est montré ci-dessous.

<img src="img/pseudocode_policyiteration.png" width="600" height="250" align="center"/>

<center><em><span style="color:gray">Sutton and Barto, chapitre 4</span></em></center>

<center><em><span style="color:gray">Ici, l'algorithme s'arrête si la nouvelle policy et la précédente sont égales. Cependant, il est possible qu'il existe plusieurs policy optimales $\pi_*$ , ce qui rendrait cet algorithme bloqué entre deux ou plus policy optimales. Il ne peut y avoir par contre qu'une seule value function $v_*$.</span></em></center>

# Exemple Gridworld

In [1]:
import numpy as np

import envs.gridworld_dennybritz as grd

In [2]:
env = grd.GridworldEnv()

### Helper function

#### Cette fonction sert d'interface avec l'environnement.
* `compute_q_value_for_s_a(env, V, s, a, gamma)` retourne, pour le state $s$ et l'action $a$ spécifiés la valeur $Q(s,a)$ soit $\sum_{s'} p(s'|s,a) [R_{ss'}^a + \gamma V(s')]$. <br>Cette fonction interroge la fonction `P[s][a]` de l'environnement qui renvoit une liste de tuple de la forme : `(p(s'|s,a), s', r(s,a,s'), done?)`. L'algorithme loop sur ces tuples et ajoute à `q` (équivalent à $Q(s,a)$) la valeur $p(s'|s,a) [R_{ss'}^a + \gamma V(s')]$.

In [3]:
def compute_q_value_for_s_a(env, V, s, a, gamma):
    q = 0
    
    for p_sPrime, sPrime, r_ss_a, done in env.P[s][a]:
        q += p_sPrime * (r_ss_a + gamma * V[sPrime])
        
    return q

### Evaluation and improvement functions

#### Ces deux fonctions servent respectivement d'évaluer une policy et de l'improve.
* `evaluate_policy(env, pi, V, gamma, theta)` reprend la même structure que vu précédemment. Elle retourne en plus le boolean `improved`, qui détermine si il y a eu en changement entre la value function originale et la nouvelle.
* `improve_policy(env, pi, V, gamma)` retourne la policy improved.

In [4]:
def evaluate_policy(env, pi, V, gamma, theta):
    #env : OpenAI-like env
    #pi : [nS * nA] matrix giving for each state a probability distribution over all actions
    #V : [nS * 1] column vector of state values
    #gamma : discount factor, e [0, 1]
    #theta : threshold
    
    #returns V : [nS * 1] column vector of updates state values
    #        improved : whether or not V has been improved
    
    V_updated = np.copy(V)
    improved = True
    
    while True:        
        delta = 0
        for s in range(env.nS):
            V_new = 0
        
            for a in range(env.nA):
                prob_a = pi[s][a]
                q_s_a = compute_q_value_for_s_a(env, V, s, a, gamma)
            
                V_new += prob_a * q_s_a
        
            delta = max(delta, np.abs(V_new - V_updated[s]))
            V_updated[s] = V_new

        if(delta < theta):
            break
    
    if(np.array_equal(V, V_updated)):
        improved = False
    
    return V_updated, improved

def improve_policy(env, pi, V, gamma):
    #env : OpenAI-like env
    #pi : [nS * nA] matrix giving for each state a probability distribution over all actions
    #V : [nS * 1] column vector of state values
    #gamma : discount factor, e [0, 1]
    #theta : threshold
    
    #returns pi : updated pi
    
    for s in range(env.nS):
        q_s = np.zeros([env.nA, 1])
    
        for a in range(env.nA):
            q_s[a] = compute_q_value_for_s_a(env, V, s, a, gamma)
        
        best_a = np.argmax(q_s)
        pi[s] = np.eye(env.nA)[best_a]
    
    return pi

### Initialization

La policy $\pi$ et la value function $V$ sont initalizées à des valeurs random.

Les hyperparamètres $\gamma$ and $\theta$ sont également initializés. Le discount factor prend la valeur de 1, cela ne posera pas de problème car l'horizon est dans ce cas là fini. Le threshold $\theta$ détermine l'écart maximal entre deux mises à jour de value function à partir duquel l'algorithme aura considéré qu'il a convergé sur $v_\pi$.

In [5]:
pi = np.ones([env.nS, env.nA]) * 0.25
V = np.zeros([env.nS, 1])

gamma = 1.0
theta = 0.00001

### Iterations

A chaque iteration $k$, l'algorithme performe une évaluation et un improvement de $\pi$. Dès que la value function a convergé, l'algorithme s'arrête.

In [6]:
k = 0
while True:
    k+=1

    V, improved = evaluate_policy(env, pi, V, gamma, theta)
    pi = improve_policy(env, pi, V, gamma)
    
    if(improved == False):
        print("Finished after " + str(k) + " iterations.")
        break

Finished after 4 iterations.


In [7]:
V, pi

(array([[ 0.],
        [-1.],
        [-2.],
        [-3.],
        [-1.],
        [-2.],
        [-3.],
        [-2.],
        [-2.],
        [-3.],
        [-2.],
        [-1.],
        [-3.],
        [-2.],
        [-1.],
        [ 0.]]), array([[1., 0., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 0., 1., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.]]))

La policy $\pi_*$ ont été trouvé par l'algorithme :

<img src="img/gridworld_actions.png" width="250" height="250" align="center"/>

Ainsi que que la value function $v_*$ qui, de par la nature du problème, représente le nombre de steps moyen avant la fin de l'épisode, en suivant la policy $\pi$.

<img src="img/gridworld_states_values_best.png" width="250" height="250" align="center"/>

learn more:
* http://incompleteideas.net/book/first/ebook/node46.html
* http://billy-inn.github.io/blog/2016/10/06/notes-on-reinforcement-learning-2-dynamic-programming/

# + Modified Policy Iteration

L'algorithme de policy iteration a-t-il vraiment besoin que $V$ converge exactement à $v_\pi$ pour ensuite improve la policy ?

Le Modified Policy Iteration (MPI) consiste à ne répéter seulement $k$ fois la mise à jour de $V(s)$, au lieu de la répéter jusqu'à que $V$ converge (ce que fait Policy Iteration). En effet, il est raisonnable de penser que seulement $k$ mises à jour suffisent à déterminer les meilleurs actions pour la policy.

<font size="4">Modified Policy Iteration Pseudocode</font>

repeat, while $V$ stops updating:

&emsp;<strong>Evaluation</strong><br/>
&emsp;repeat $k$ times:<br/>
&emsp;&emsp; for each $s$:<br/>
&emsp;&emsp;&emsp; $V(s) \leftarrow \sum_{a} \pi(a|s) \sum_{s'} p(s'|s,a) [R_{ss'}^a + γ V(s')]$

&emsp;<strong>Improvement</strong><br/>
&emsp;for each $s$:<br/>
&emsp;&emsp;&emsp; $\pi(s) \leftarrow argmax_a [\sum_{s'} p(s'|s,a) [R_{ss'}^a + γ V(s')]]$

Il se trouve que si $k=1$, alors cela revient au même que l'algorithme de Value Iteration, le second algorithme très utilisé pour résoudre un problème model-based.